In [ ]:
from Capsule.Layer.CapsuleLayer import CapsuleRouting
import torch
routing = CapsuleRouting(mode='em')

v = torch.rand(2, 8, 10, 16, 1, 1)
a = torch.rand(2, 8, 1, 1)
v, a = routing(v, a)
print(a)

In [ ]:
from Capsule.Layer.CapsuleLayer import AdaptiveCapsuleHead
import torch

model = AdaptiveCapsuleHead(32, 10, 4, 1, False, *["dynamic", 10, 1.5])
v = torch.rand(2, 32, 5, 5)

out, v = model(v, get_capsules=True)
print(v.shape)
print(v)

In [ ]:
import yaml
import torch
from Capsule.Layer.Classifier import CapsuleWrappingClassifier
with open('config/new-config.yml', 'r') as stream:
    PARAMS = yaml.safe_load(stream)
    print(PARAMS)
model = CapsuleWrappingClassifier(model_configs=PARAMS['architect_settings'])

img = torch.rand(2, 3, 224, 224)

out = model(img)
print(out.shape)
print(out)

In [ ]:
from Capsule.ultis import Cub2011
from matplotlib import pyplot as plt
from torchvision.transforms import ToPILImage
dataset = Cub2011(mode='test', data_path='data/CUB_200_2011', transform=None)

img, label, oimg = dataset[10]
print(label)
plt.imshow(ToPILImage()(oimg))
plt.show()

In [ ]:
import numpy as np
a = np.arange(60.).reshape(3,4,5)
b = np.arange(24.).reshape(4,3,2)
print(a)
print(b)
np.einsum('ijk,jil->kl', a, b)

In [ ]:
import torch
A = torch.rand(2, 15, 4, 4, 3, 3)
B = torch.rand(15, 4, 4, 8)
v = torch.einsum('bBijHW, BjkC -> bBCikHW', A, B)
print(v.shape)

## Linear Classifier

In [ ]:
PARAMS =  {
    "architect_settings" : {
            "task": "None",
            "name": "model-test",
            "backbone": {
                    "name": "resnet-s",
                    "is_full": False,
                    "is_pretrained": True,
                    "is_freeze": True, 
            },
            "n_cls": 2,
            "is_caps": False,
            "caps":{
                "mode": 1,
                "cap_dims": 4,
                "routing":{
                        "type": "dynamic",
                        "params": [3, 0.01, 1.5]
                    }
                }
            },
    "dataset_settings": {
            
            },
    "training_settings":{
    
    }
}

from Capsule.Layer.Classifier import CapsuleWrappingClassifier
from Capsule.ultis import Cub2011
from torch.utils.data import DataLoader
import torch.nn.functional as F
import h5py
import numpy as np

model = CapsuleWrappingClassifier(model_configs=PARAMS["architect_settings"])
model.eval()

def get_train_data(model, dataloder):
    train_data = []
    train_label = []    
    for batch in dataloder:
        img, label, oimg = batch
        feats = model.backbone(img)
        out = feats.reshape(-1, 512, 7, 7)
        train_data.append(out.detach().numpy())
        train_label.append(label.detach().numpy())

    train_data = np.concatenate(train_data, axis=0)
    train_label = np.concatenate(train_label, axis=0)
    print(train_data.shape)
    print(train_label.shape)
    return train_data, train_label
    

In [ ]:
dataset = Cub2011(mode="train", data_path="data/CUB_200_2011")
dataloder = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)

train_data, train_label = get_train_data(model, dataloder)

hf = h5py.File('train_data.h5', 'w')
hf.create_dataset('data', data=train_data)
hf.create_dataset('label', data=train_label)
hf.close()

In [ ]:
dataset = Cub2011(mode="test", data_path="data/CUB_200_2011")
dataloder = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)

test_data, test_label = get_train_data(model, dataloder)

hf = h5py.File('test_data.h5', 'w')
hf.create_dataset('data', data=test_data)
hf.create_dataset('label', data=test_label)
hf.close()

In [ ]:
hf = h5py.File('data/CUB_ResNet_train_data.h5', 'r')
train_data = hf['data']
train_label = hf['label']
hf.close()
print(train_data)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import h5py

hf = h5py.File('data/CUB_ResNet_train_data.h5', 'r')
train_data = np.array(hf['data'])
train_label = np.array(hf['label'])
hf.close()
hf = h5py.File('data/CUB_ResNet_test_data.h5', 'r')
test_data = np.array(hf['data'])
test_label = np.array(hf['label'])
hf.close()

train_data = np.mean(train_data, axis=(2, 3))
test_data = np.mean(test_data, axis=(2, 3))

#scikit-learn MLPClassifier on train_data, train_label
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(512, 512), max_iter=1000)
clf.fit(train_data, train_label)
accuracy_score(clf.predict(test_data), test_label)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='rbf')
clf.fit(train_data, train_label)
accuracy_score(clf.predict(train_data), train_label)
accuracy_score(clf.predict(test_data), test_label)

In [2]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import h5py
import numpy as np

hf = h5py.File('data/CUB_ResNet_train_data.h5', 'r')
train_data = np.array(hf['data'])
train_label = np.array(hf['label'])
hf.close()

tensor_x = torch.Tensor(train_data) # transform to torch tensor
tensor_y = torch.Tensor(train_label)
my_dataset = TensorDataset(tensor_x,tensor_y)
train_loader = DataLoader(my_dataset, batch_size=32, shuffle=True)

In [3]:
hf = h5py.File('data/CUB_ResNet_test_data.h5', 'r')
test_data = np.array(hf['data'])
test_label = np.array(hf['label'])
hf.close()

tensor_x = torch.Tensor(test_data) # transform to torch tensor
tensor_y = torch.Tensor(test_label)
my_val_dataset = TensorDataset(tensor_x,tensor_y)
valid_loader = DataLoader(my_val_dataset, batch_size=32, shuffle=True)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from Capsule.Layer.CapsuleLayer import AdaptiveCapsuleHead

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

classifier = nn.Sequential(
                AdaptiveCapsuleHead(512, 200,
                4, 4, True, 
                *['dynamic', 3, 2]),
                # nn.LogSoftmax(dim=1),
                nn.Flatten(start_dim=1)
            )

# Define the classifier on top of the ResNet
# classifier = nn.Sequential(
#     nn.Linear(512, 512, 1),
#     nn.ReLU(),
#     nn.Linear(512, 512, 1),
#     # nn.LogSoftmax(dim=1),
#     # nn.Flatten(start_dim=1)
# )

# # Combine the ResNet and classifier
# from torchvision.models import resnet18
# resnet = resnet18(pretrained=True)
# resnet.fc = nn.Identity()
# model = nn.Sequential(resnet, classifier)
model = classifier.to(device)
# Load the MNIST dataset
# train_dataset = Cub2011(mode="train", data_path="data/CUB_200_2011")
# valid_dataset = Cub2011(mode="val", data_path="data/CUB_200_2011")

# # Set hyperparameters
batch_size = 64
learning_rate = 0.0001
num_epochs = 100

# # Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

max_val = 0
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train_samples = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/ {num_epochs} - Training"):
        
        images = images.to(device)
        labels = labels.to(device).long()
        # print(images, labels)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

        train_loss += loss.item()

    # Compute average training accuracy and loss
    train_accuracy = 100.0 * train_correct / total_train_samples
    train_loss /= len(train_loader)

    # Validation phase
    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    total_valid_samples = 0

    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc=f"Epoch {epoch + 1}/ {num_epochs} - Validation"):
            # Forward pass
            images = images.to(device)
            labels = labels.to(device).long()
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Compute validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            valid_correct += (predicted == labels).sum().item()
            total_valid_samples += labels.size(0)

            valid_loss += loss.item()

    # Compute average validation accuracy and loss
    valid_accuracy = 100.0 * valid_correct / total_valid_samples
    valid_loss /= len(valid_loader)
    if(valid_accuracy > max_val):
        max_val = valid_accuracy
    # Print epoch-wise results
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Valid Loss: {valid_loss:.4f} | Valid Accuracy: {valid_accuracy:.2f}%")
print(f"Max Valid Accuracy: {max_val:.2f}%")

cuda:1


Epoch 1/ 100 - Validation: 100%|██████████| 182/182 [00:00<00:00, 466.30it/s]



Epoch 1/100
Train Loss: 5.0570 | Train Accuracy: 4.45%
Valid Loss: 4.7629 | Valid Accuracy: 10.13%


Epoch 2/ 100 - Validation: 100%|██████████| 182/182 [00:00<00:00, 468.07it/s]



Epoch 2/100
Train Loss: 4.5052 | Train Accuracy: 14.21%
Valid Loss: 4.3443 | Valid Accuracy: 16.95%


...


Epoch 98/100
Train Loss: 0.3988 | Train Accuracy: 92.46%
Valid Loss: 3.1302 | Valid Accuracy: 44.60%


Epoch 99/ 100 - Validation: 100%|██████████| 182/182 [00:00<00:00, 466.99it/s]



Epoch 99/100
Train Loss: 0.3214 | Train Accuracy: 95.65%
Valid Loss: 2.9316 | Valid Accuracy: 48.96%


Epoch 100/ 100 - Validation: 100%|██████████| 182/182 [00:00<00:00, 461.20it/s]


Epoch 100/100
Train Loss: 0.2116 | Train Accuracy: 98.47%
Valid Loss: 2.8911 | Valid Accuracy: 49.71%
Max Valid Accuracy: 52.50%


- CUB-resnet-sci-SVM: 0.5904
- CUB-resnet-freeze-2hMLP: 0.5856
- CUB-resnet-2hMLP: 0.5188
- CUB-resnet-sci-MLP: 0.577